<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Medmcqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datacamp.com/tutorial/mistral-7b-tutorial

# DEPENDENCIES

In [2]:
import torch
import tensorboard

In [3]:
# Install Pytorch & other libraries
#!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


In [4]:
# versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.9, 0.7.10, 0.7.11, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.9.2, 0.9.3, 0.9.4)
#ERROR: No matching distribution found for trl==0.0.99
! pip install trl==0.8.6 -q

In [5]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [8]:
# set device
device = 'cuda'

In [9]:
torch.__version__

'2.3.0+cu121'

In [10]:
#!apt-get update && apt-get install -y cuda-11-0

In [11]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Mon Jun 10 07:28:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0       

In [12]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

# Data loading


https://huggingface.co/datasets/sakharamg/AviationQA

In [13]:
! pip install datasets ninja packaging --quiet

In [14]:
from datasets import load_dataset
print("Preprocessing dataset Medmcqa")
dataset = load_dataset("openlifescienceai/medmcqa")

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["validation"].to_json("validation_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Preprocessing dataset Medmcqa


Creating json from Arrow format:   0%|          | 0/183 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

1910937

In [15]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 4183
    })
})


In [16]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
print(dataset)

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})


In [18]:
train_question=dataset['question']
train_answer=dataset['exp']

In [19]:
!rm -rf /content/summary-qa.txt
filename='/content/summary-qa.txt'

import re

# python object to be appended
for n in range(10000):
    if train_answer[n] == None:
       train_answer[n] = 'Not possible to get or use'
    string = train_answer[n]
    #result = string.replace("\nRef", "-Ref")
    str_question=train_question[n]
    question= re.sub("\n", "", str_question)
    answer = re.sub("\n", "", string)
    #print(answer)
    if len(answer)==0:
        answer='Not possible to get or use'
    text='<s>[INST] %s [/INST] %s </s>'%(question,answer)
    #print(text)
    with open(filename, 'a') as f:
        f.write(text + '\n')

In [20]:
train_question[18]

'The process of hardening a cement matrix through hydration with oral fluids  to achieve greater mechanical strength is known as:'

In [21]:
train_answer[18]

'Maturation (cement)—The process of hardening a cement matrix through hydration with oral fluids\xa0to achieve greater mechanical strength.\nRef: Phillip’s Science of Dental Materials ed 12 pg 309'

In [22]:
text0 = load_dataset("text", data_files="/content/summary-qa.txt", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
text0

Dataset({
    features: ['text'],
    num_rows: 10000
})

In [24]:
nrec=10000
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['question'][0:nrec]
dataset_final_Answer=dataset['exp'][0:nrec]
dataset_final_text=text0[0:nrec]['text']

In [25]:
dataset[84]

{'id': 'fc374693-12de-4ca4-8854-34bd1a9c686b',
 'question': 'During extraction of the upper first molar, the mesio buccal root is missing and is suspected to have been pushed into the maxillary sinus.\nThe best position for the patient immediately after the incident is',
 'opa': 'To position the patient upright',
 'opb': 'To position the patients in a semi inclined posture to visualize the perforation',
 'opc': 'Trendelenberg position',
 'opd': 'Reverse trendelenberg position',
 'cop': 0,
 'choice_type': 'single',
 'exp': None,
 'subject_name': 'Surgery',
 'topic_name': None}

In [26]:
text0[84]

{'text': '<s>[INST] During extraction of the upper first molar, the mesio buccal root is missing and is suspected to have been pushed into the maxillary sinus.The best position for the patient immediately after the incident is [/INST] Not possible to get or use </s>'}

In [27]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer
datasetF['text'] = dataset_final_text

In [28]:
datasetF

,id,Question,Answer,text
0,e9ad821a-c438-4965-9f77-760819dfa155,Chronic urethral obstruction due to benign pri...,Chronic urethral obstruction because of urinar...,<s>[INST] Chronic urethral obstruction due to ...
1,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,Which vitamin is supplied from only animal sou...,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...,<s>[INST] Which vitamin is supplied from only ...
2,5c38bea6-787a-44a9-b2df-88f4218ab914,All of the following are surgical options for ...,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba...",<s>[INST] All of the following are surgical op...
3,cdeedb04-fbe9-432c-937c-d53ac24475de,Following endaerectomy on the right common car...,The central aery of the retina is a branch of ...,<s>[INST] Following endaerectomy on the right ...
4,dc6794a3-b108-47c5-8b1b-3b4931577249,Growth hormone has its effect on growth through?,"Ans. is 'b' i.e., IGI-1GH has two major functi...",<s>[INST] Growth hormone has its effect on gro...
...,...,...,...,...
9995,94ccf793-8bea-4e15-9033-ec1279f1c5ef,All of the following results in central nervou...,Hypocalcemia can occur in chronic renal failur...,<s>[INST] All of the following results in cent...
9996,4d6ab1ac-4d5e-4b3b-a1ba-5c487ecfc55c,Which of the following statement about DDT is ...,"Ans. is `a' i.e., Immediately kills the prey D...",<s>[INST] Which of the following statement abo...
9997,56c0aa12-bb49-451e-843b-efce0714cb7d,All are true regarding METHANOL poisoning except,Fomepizole is a alcohol dehydrogenase inhibito...,<s>[INST] All are true regarding METHANOL pois...
9998,30af852c-9cf9-45fd-a814-756e3ccbaf4a,An 18-year-old male presented with acute onset...,Botulism is caused by neurotoxins of Clostridi...,<s>[INST] An 18-year-old male presented with a...


In [29]:
datasetF['Question'][84]

'During extraction of the upper first molar, the mesio buccal root is missing and is suspected to have been pushed into the maxillary sinus.\nThe best position for the patient immediately after the incident is'

In [30]:
datasetF['Answer'][84]

In [31]:
datasetF['text'][84]

'<s>[INST] During extraction of the upper first molar, the mesio buccal root is missing and is suspected to have been pushed into the maxillary sinus.The best position for the patient immediately after the incident is [/INST] Not possible to get or use </s>'

In [32]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [33]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       1.8Gi       3.9Gi        12Mi        77Gi        80Gi
Swap:             0B          0B          0B


# Loading the Mistral 7B model

In [34]:
!pip install mistral_inference -q

In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024, 08 MARCH 2024 and 27 MAY 2024

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    is_decoder=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
#model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

Inference

Build an inference pipeline with tokenizer and the model.

In [37]:
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [38]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt0="What is the capital of russia?"
#prompt="What are some interesting sites to visit in the Bay Area?"

prompt = f"Instruct: Find THE answer for the following  question.\n{prompt0}\nOutput:\n"
#prompt = f"Instruct: Find only the answer for the following  question.\n{prompt0}"

#prompt="What are some interesting sites to visit in the Bay Area?"
#prompt="What is the capital of russia?"
outputs = pipe(prompt0, max_new_tokens=128, temperature=0.9,do_sample=True,top_k=30, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

print('Question: %s'%prompt0)
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt0):].strip()}")


Question: What is the capital of russia?

Generated Answer:
2013-11-25 08:27:18

Answer: Moscow, Russia


In [39]:
print(outputs[0]['generated_text'])

What is the capital of russia?

2013-11-25 08:27:18

Answer: Moscow, Russia


In [40]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [41]:
#index=321458
#index=10000
#features: ['id', 'Question', 'Answer'],
index=2
prompt = datasetF['id'][index]
print(prompt)

prompt = datasetF['Question'][index]
print(prompt)

prompt = datasetF['Answer'][index]
print(prompt)

prompt = datasetF['text'][index]
print(prompt)

5c38bea6-787a-44a9-b2df-88f4218ab914
All of the following are surgical options for morbid obesity except -
Ans. is 'd' i.e., Roux en Y Duodenal Bypass Bariatric surgical procedures include:a. Vertical banded gastroplastyb. Adjustable gastric bandingc. Roux-en Y gastric bypass (Not - Roux-en Y Duodenal Bypass)d. Biliopancreatic diversione. Duodenal switcho The surgical treatment of morbid obesity is known as bariatric surgery.o Morbid obesity is defined as body mass index of 35 kg/m2 or more with obesity related comorbidity, or BMI of 40 kg/m2 or greater without comorbidity.o Bariatric operations produce weight loss as a result of 2 factors. One is restriction of oralintake. The other is malabsorbtion of ingested food.o Gastric restrictive procedures include Vertical banded gastroplasty & Adjustable gastric bandingo Malabsorbtive procedures include Biliopancreatic diversion, and Duodenal switcho Roux-en Y gastric bypass has features of both restriction and malabsorptionBariatric Operati

MODEL GENERATION - ZERO SHOT

In [42]:
index=2

#features: ['id', 'Question', 'Answer'],
#prompt = dataset[index]['Question']
#summary = dataset[index]['Answer']

prompt = datasetF['Question'][index]
summary = datasetF['Answer'][index]

original_model = model

formatted_prompt = f"Instruct: provide an answer for the following dialog.\n{prompt}"

res = original_model.generate(tokenizer.encode(formatted_prompt, return_tensors="pt"), max_length=128, do_sample=False)
output = tokenizer.decode(res[0], skip_special_tokens=True)

#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to('cuda') # Assuming you want to run on GPU
#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to(torch.bfloat16)
#res = model.generate(input_ids, max_length=512, do_sample=False)
#output = tokenizer.decode(res[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [43]:
dash_line = '-'.join('' for x in range(100))
print(dash_line)

print(f'Dialogue :\n{prompt}')
print(dash_line)

print(f'ACTION BY THE AGENT-HUMAN:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Dialogue :
All of the following are surgical options for morbid obesity except -
---------------------------------------------------------------------------------------------------
ACTION BY THE AGENT-HUMAN:
Ans. is 'd' i.e., Roux en Y Duodenal Bypass Bariatric surgical procedures include:a. Vertical banded gastroplastyb. Adjustable gastric bandingc. Roux-en Y gastric bypass (Not - Roux-en Y Duodenal Bypass)d. Biliopancreatic diversione. Duodenal switcho The surgical treatment of morbid obesity is known as bariatric surgery.o Morbid obesity is defined as body mass index of 35 kg/m2 or more with obesity related comorbidity, or BMI of 40 kg/m2 or greater without comorbidity.o Bariatric operations produce weight loss as a result of 2 factors. One is restriction of oralintake. The other is malabsorbtion of ingested food.o Gastric restrictive procedures include Vertical banded gastroplasty & 

In [44]:
print(len(datasetF))

10000


In [45]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

# Adding the adopter to the layer

In [46]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

# Hyperparmeters

In [47]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="Mistral-7B-v0.1_MedmcQA", # 06/06/2024 #dataset_AviationQA

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=10,                     # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

https://huggingface.co/docs/trl/en/sft_trainer

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_text_to_SQL.ipynb

In [48]:
#print(dataset.column_names)
print(datasetF.columns)

Index(['id', 'Question', 'Answer', 'text'], dtype='object')


In [49]:
print(datasetF['Answer'][2])

Ans. is 'd' i.e., Roux en Y Duodenal Bypass Bariatric surgical procedures include:a. Vertical banded gastroplastyb. Adjustable gastric bandingc. Roux-en Y gastric bypass (Not - Roux-en Y Duodenal Bypass)d. Biliopancreatic diversione. Duodenal switcho The surgical treatment of morbid obesity is known as bariatric surgery.o Morbid obesity is defined as body mass index of 35 kg/m2 or more with obesity related comorbidity, or BMI of 40 kg/m2 or greater without comorbidity.o Bariatric operations produce weight loss as a result of 2 factors. One is restriction of oralintake. The other is malabsorbtion of ingested food.o Gastric restrictive procedures include Vertical banded gastroplasty & Adjustable gastric bandingo Malabsorbtive procedures include Biliopancreatic diversion, and Duodenal switcho Roux-en Y gastric bypass has features of both restriction and malabsorptionBariatric Operations: Mechanism of ActionRestrictiveVertical banded gastroplastyLaparoscopic adjustable gastric bandingLarge

# Supervised fine-tuning (SFT)  parameters

In [50]:
!pip install trl -q

In [51]:
from trl import SFTTrainer
import datasets

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)

max_seq_length = 3072 # max sequence length for model and packing of the dataset

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
#model.to(device)


trainer = SFTTrainer(
    model=original_model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    #device_map={"": device},
    #formatting_func=formatting_func,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        ##"per_device_train_batch_size": 8,
        #"per_device_eval_batch_size": 8,
        #"num_train_epochs": 3.0},
    }
)

# Model training AND Saving the fine-tuned model

In [ ]:
#del device
#model.to(torch.device('cuda'))
trainer.train()

# /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py
#  1406

# save model
trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.070700
20,1.877200
30,1.799800
40,1.817900
50,1.798600
60,1.781500
70,1.789900
80,1.771800
90,1.618100
100,1.221800


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

In [ ]:
# free the memory again
del model
del original_model
del trainer
torch.cuda.empty_cache()

# Model evaluation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

In [ ]:
!ls -ltha /content/Mistral-7B-v0.1_AviationQA/

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
peft_model_id = "/content/Mistral-7B-v0.1_MedmcQA"


# Load Model with PEFT adapter
modelnew = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)


# load into pipeline
generation_pipeline = pipeline("text-generation", model=modelnew, tokenizer=tokenizer)

In [ ]:
prompt='capital of russia?'
outputs = generation_pipeline(prompt, max_new_tokens=512, do_sample=True, temperature=0.1,
                              top_k=50, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

In [ ]:
print(outputs[0]['generated_text'])

#print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")